In [12]:
import wave
import struct
from scipy import *
from __future__ import division
from pylab import *
from numpy import *

def clearsign(clear_sign):
    maxx = max(clear_sign[1:])

    for i in range(1, len(clear_sign)):
        if clear_sign[i] < maxx/2:
            clear_sign[i] = 0

def opens(file_name):
    file       = wave.open(file_name, "r")
    channel, samp_width, frame_rate, frames_number, type_exam, comp_nam = file.getparams()
    frame      = file.readframes(frames_number * channel)
    decomp     = struct.unpack_from("%dh" % frames_number * channel, frame)
    
    if channel == 2:
        lchannel = array(decomp[0::2], dtype='float_')
        lchannel /= 2
        rchannel = array(decomp[1::2], dtype='float_')
        rchannel /= 2
        sign = rchannel + lchannel
    else:
        sign = array(decomp, dtype='float_')
    
    wid = frame_rate
    return wid, sign

def cepstrum(sign, k, wid):
    a = []
    
    t = 200
    width = int(wid*t / 1000)
    windows_number = int(k / width)

    
    for i in range(windows_number):
        order   = i * width
        sign_t  = sign[order:order+width]
        sign_t *= blackman(len(sign_t))
        sign1   = rfft(sign_t)
        sign1   = abs(sign1)
        clearsign(sign1)
        
        for i in range(0, len(sign1)):
            if sign1[i] > 0:
                sign1[i] = log(sign1[i])

        sign2   = irfft(sign1)
        sign2   = abs(sign2)

        order   = 1
        scope1  = int(math.floor(wid/60))
        scope2  = int(math.floor(wid/200))
        j = max(sign2[scope2:scope1])
        
        for i in range(scope2, scope1):
            if j == sign2[i]:
                order = i
                break

        a.append(wid/order)

    answer = np.median(sorted(a))
    return answer

def main():
    file = "059_K.wav"
    
    wid, sign = opens(file)
    k = len(sign)
    a = cepstrum(sign, k, wid)

    if a <= 165:
        print('M')
    else:
        print('K')
        
if __name__ == "__main__":
   main()

K
